In [1]:
import pandas as pd

In [2]:
# Membaca file Excel dengan banyak sheet
file_path = 'data/dataset_primer_2.xlsx' 
df_sheets = pd.read_excel(file_path, sheet_name=None)


In [3]:
# Fungsi untuk menangani outlier menggunakan IQR
def handle_outliers(df, columns):
    for col in columns:
        # Menghitung Q1, Q3, dan IQR untuk kolom
        Q1 = df[col].quantile(0.25)
        Q3 = df[col].quantile(0.75)
        IQR = Q3 - Q1

        # Menentukan batas bawah dan atas untuk outlier
        lower_bound = Q1 - 1.5 * IQR
        upper_bound = Q3 + 1.5 * IQR

        # Ganti nilai outlier dengan batas bawah atau atas yang sesuai
        df.loc[df[col] < lower_bound, col] = Q1
        df.loc[df[col] > upper_bound, col] = Q3

    return df

In [4]:
for sheet_name, df in df_sheets.items():
    print(f"Memproses sheet: {sheet_name}")

    # Mengidentifikasi kolom yang memiliki prefiks 'berat'
    berat_columns = [col for col in df.columns if col.startswith('berat_')]

    # Mengisi data kosong pada kolom yang memiliki prefiks 'berat_' menggunakan interpolasi linear
    for col in berat_columns:
        # Mengisi nilai kosong menggunakan interpolasi linear terlebih dahulu
        df[col] = df[col].interpolate(method='linear', axis=0)

        # Jika masih ada nilai kosong di awal data (baris pertama), gunakan forward fill untuk mengisinya
        if df[col].isnull().iloc[0]:  # Cek jika baris pertama kosong
            df.loc[0, col] = df.loc[1, col]  # Mengisi dengan nilai baris kedua (forward fill manual)

    # Menangani outlier pada kolom 'berat_*'
    df = handle_outliers(df, berat_columns)

    # Menyimpan kembali sheet yang telah diproses
    df_sheets[sheet_name] = df

Memproses sheet: day 05
Memproses sheet: day 06
Memproses sheet: day 07
Memproses sheet: day 08
Memproses sheet: day 09
Memproses sheet: day 10
Memproses sheet: day 12
Memproses sheet: day 13
Memproses sheet: day 14
Memproses sheet: day 15
Memproses sheet: day 16
Memproses sheet: day 19


In [5]:
# Menyimpan hasil pengolahan ke file Excel baru dengan path absolut
output_file_path = 'data/data_processed.xlsx'  # Ganti dengan path yang sesuai
with pd.ExcelWriter(output_file_path, engine='openpyxl') as writer:
    for sheet_name, df in df_sheets.items():
        df.to_excel(writer, sheet_name=sheet_name, index=False)